# Статанализ масок облаков и снега

В файл были экспортированы обучающие данные по облачности и снегу (см. https://176.9.38.120/cruncher/notebooks/deforestation/CMask_trainings.ipynb).

In [1]:
points = read.table('data/CMask_trainings/random_points.csv', header=TRUE, sep=';')

# summary(points)

Содержимое колонок:

  * cat -- идентификатор точки, в которой производились измерения;
  * l1_b1 -- l1_b11 -- данные LANDSAT8 (после атмосферной коррекции) по сцене LC81120282015365LGN00;
  * l2_b1 -- l2_b11 -- данные LANDSAT8 (после атмосферной коррекции) по сцене LC81130272015356LGN00;
  * t1, t2 -- данные по наземным объектам сцен LC81120282015365LGN00 и LC81130272015356LGN00 соответственно, значения: 1-облака;2-снег;3-остальное.
  * x, y -- координаты (UTM);

Соберем данные в dataframe для удобства анализа так, чтобы в нем одна строка соответствовала одному измерению, а не двум, как сейчас:

In [2]:
tmp1 = points[c('cat', 'x', 'y', 't1',
                'l1_b1', 'l1_b2', 'l1_b3', 'l1_b4', 'l1_b5', 'l1_b6', 
                'l1_b7', 'l1_b8', 'l1_b9', 'l1_b10', 'l1_b11'
               )]
names(tmp1) = c('cat', 'x', 'y', 't',
                'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11'
               )

tmp1$scene = 'S1' # 'LC81120282015365LGN00'

tmp2 = points[c('cat', 'x', 'y', 't2',
                'l2_b1', 'l2_b2', 'l2_b3', 'l2_b4', 'l2_b5', 'l2_b6', 
                'l2_b7', 'l2_b8', 'l2_b9', 'l2_b10', 'l2_b11'
               )]
names(tmp2) = c('cat', 'x', 'y', 't',
                'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11'
               )
tmp2$scene = 'S2' # 'LC81130272015356LGN00'


points = rbind(tmp1, tmp2)

points$t[points$t==0] = NA

points$scene = factor(points$scene)
points$t = factor(points$t)
levels(points$t) <- list(Cloud=1, Snow=2, Other=3)

points = na.omit(points)

summary(points)

      cat              x                y               t        
 Min.   :    1   Min.   :434113   Min.   :4979683   Cloud: 1579  
 1st Qu.:12508   1st Qu.:496221   1st Qu.:5060181   Snow : 3644  
 Median :24998   Median :536605   Median :5117104   Other:45316  
 Mean   :24998   Mean   :541089   Mean   :5112347                
 3rd Qu.:37496   3rd Qu.:586255   3rd Qu.:5165598                
 Max.   :50000   Max.   :664168   Max.   :5215814                
       b1               b2                b3                b4         
 Min.   :0.1124   Min.   :0.08453   Min.   :0.04529   Min.   :0.02953  
 1st Qu.:0.2011   1st Qu.:0.16832   1st Qu.:0.11290   1st Qu.:0.09587  
 Median :0.2291   Median :0.19891   Median :0.14421   Median :0.13350  
 Mean   :0.2529   Mean   :0.22642   Mean   :0.17388   Mean   :0.17051  
 3rd Qu.:0.2732   3rd Qu.:0.24862   3rd Qu.:0.19710   3rd Qu.:0.19912  
 Max.   :0.9407   Max.   :1.07539   Max.   :1.17382   Max.   :1.45950  
       b5               b6        

Видим, что в 9-м канале после атмосферной коррекции получились отрицательные значения. Объясняется это, по всей видимости, тем, что используемые в ходе коррекции константы (gain или bias, см. метаданные) содержали неподходящие для данной сцены значения. Аналогично, почти во всех каналах значения отражающей способности оказались выше 1.

In [3]:
# Число примеров, оставшихся после удаления отсутсвующих данных
dim(points)

[1] 50539    16

In [4]:
points$b8[points$b9 < 0] = 0.0

points$b2[points$b2 > 1] = 1.0
points$b3[points$b3 > 1] = 1.0
points$b4[points$b4 > 1] = 1.0
points$b5[points$b5 > 1] = 1.0
points$b8[points$b8 > 1] = 1.0

## Первый взгляд на данные

### Отражающая способность различных объектов

Построим ящики с усами по каждому каналу и сравним какие значения отражающей способности типичны для каждого типа поверхности.

In [5]:
opar = par(no.readonly = TRUE)      # make a copy of current settings

png("Img/CMASK/type_vs_bands.png", width=728, height=728, units="px")
    par(mfrow = c(3,4))
    boxplot(b1 ~ t, data=points, ylab='b1')
    boxplot(b2 ~ t, data=points, ylab='b2')
    boxplot(b3 ~ t, data=points, ylab='b3')
    boxplot(b4 ~ t, data=points, ylab='b4')

    boxplot(b5 ~ t, data=points, ylab='b5')
    boxplot(b6 ~ t, data=points, ylab='b6')
    boxplot(b7 ~ t, data=points, ylab='b7')
    boxplot(b8 ~ t, data=points, ylab='b8')

    boxplot(b9 ~ t, data=points, ylab='b9')
    boxplot(b10 ~ t, data=points, ylab='b10')
    boxplot(b11 ~ t, data=points, ylab='b11')
    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/type_vs_bands.png">

* 1-й--5-й и 8-й каналы ведут себя примерно одинаково: яркости пикслеей, соотвествующие снегу, выше, чем яркости облачных пикселей. А яркость пикселей облаков и снега обычно выше, чем пикселей других объектов. Однако, нельзя провести четкой границы -- области яркостей пикселей различных объектов довольно сильно пересекаются. Видно, что в этих каналах распределение имеет длинный правый хвост: на графиках видны выбросы в верхней части ящиков. Возможно, что стоит преобразовать каналы, чтобы получить более симметричное распределение. Поскольку восьмой канал - панхроматический, то он несет в основном пространственную информацию, а ничего нового с точки зрения спектральной информации не привносит. Поэтому в дальнейшем он буде опущен при анализе.
* 6-й--7-й каналы: высокое отражение в облачных областях, чуть меньшее в заснеженных и еще меньше для остальных объектов. Точно также данные показывают длинный правый хвост в распределении.
* 9-й канал: очень длинный правый хвост в облачных пикселях. Значения облачных пикселей в целом значительно выше, чем у остальных. Поскольку этот канал был разработан для исследования атмосферных эффектов, то, возможно, этот канал будет одним из основных маркеров облако/не облако.
* 10-й и 11-й каналы (температура) cхожи между собой: на них заметно, что температура облаков обычно выше, чем остальных объектов (зима?), хотя у облаков бросается в глаза очень длинный левый хвост распределения.

Преобразуем данные, чтобы получить более равномерные распределения. При этом выкинем из анализа 8-й канал, основную часть каналов прологарифмируем для избавления от длинных хвостов, 9-й канал возведем в степень, а 10-й и 11-й каналы оставим без изменений (длинный хвост только у облаков, а не у всего канала):

In [6]:
png("Img/CMASK/type_vs_logbands.png", width=728, height=728, units="px")
    par(mfrow = c(2, 4))
    boxplot(log(b1) ~ t, data=points, ylab='log(b1')
    boxplot(log(b2) ~ t, data=points, ylab='log(b2')
    boxplot(log(b3) ~ t, data=points, ylab='log(b3')
    boxplot(log(b4) ~ t, data=points, ylab='log(b4')

    boxplot(log(b5) ~ t, data=points, ylab='log(b5)')
    boxplot(log(b6) ~ t, data=points, ylab='log(b6)')
    boxplot(log(b7) ~ t, data=points, ylab='log(b7)')

    boxplot(I(b9^0.5) ~ t, data=points, ylab='sqrt(b9)')
    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/type_vs_logbands.png">

Видно, что распределения частично выровнялись, хотя местами по-прежнему остались длинные хвосты.

### Зависимость от сцены

Построим те же самые ящики с усами, но попробуем понять, насколько отличаются данные не только из-за объектов, но и из-за того, что они представлены в разных сценах.

In [7]:
png("Img/CMASK/scene_vs_logbands.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    boxplot(log(b1) ~ scene, data=points, ylab='log(b1)')
    boxplot(log(b2) ~ scene, data=points, ylab='log(b2)')
    boxplot(log(b3) ~ scene, data=points, ylab='log(b3)')
    boxplot(log(b4) ~ scene, data=points, ylab='log(b4)')

    boxplot(log(b5) ~ scene, data=points, ylab='log(b5)')
    boxplot(log(b6) ~ scene, data=points, ylab='log(b6)')
    boxplot(log(b7) ~ scene, data=points, ylab='log(b7)')

    boxplot(I(b9^0.5) ~ scene, data=points, ylab='sqrt(b9)')

    boxplot(b10 ~ scene, data=points, ylab='b10')
    boxplot(b11 ~ scene, data=points, ylab='b11')

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_vs_logbands.png">

Итак, мы видим, что сцена 2 (LC81130272015356LGN00) более яркая, чем сцена 1 (LC81120282015365LGN00) во всех каналах. Вопрос в том, насколько велика эта вариативность данных, которая вносится разными условиями съемки, и не может ли оказаться так, что условия съемки влияют на яркость пикселей гораздо больше, чем тип обеъкта (см., например, различия якркостей в 10-м и 11-м каналах).

Для того, чтобы ответить на этот вопрос построим ящики для отдельных каналов.

#### Канал B1

В первом канале видно, что яркость снега может меняться в достаточно широких пределах в зависимости от сцены, настолько, что она яркости облаков и снежных участков могут пересекаться в большой степени.

In [8]:
png("Img/CMASK/scene_type_vs_b1.png", width=728, height=728, units="px")
    boxplot(log(b1) ~ scene+t, data=points, ylab='log(b1)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b1.png">

#### Канал B2

Аналогичная картина и во втором канале.

In [9]:
png("Img/CMASK/scene_type_vs_b2.png", width=728, height=728, units="px")
    boxplot(log(b2) ~ scene+t, data=points, ylab='log(b2)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b2.png">

#### Канал B3

Аналогично и в третьем:

In [10]:
png("Img/CMASK/scene_type_vs_b3.png", width=728, height=728, units="px")
    boxplot(log(b3) ~ scene+t, data=points, ylab='log(b3)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b3.png">

#### Канал B4

In [11]:
png("Img/CMASK/scene_type_vs_b4.png", width=728, height=728, units="px")
    boxplot(log(b4) ~ scene+t, data=points, ylab='log(b4)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b4.png">

#### Канал B5

In [12]:
png("Img/CMASK/scene_type_vs_b5.png", width=728, height=728, units="px")
    boxplot(log(b5) ~ scene+t, data=points, ylab='log(b5)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b5.png">

#### Канал B6

Зато 6-й канал, похоже, меньше зависит от сцены, чем предыдущие:

In [13]:
png("Img/CMASK/scene_type_vs_b6.png", width=728, height=728, units="px")
    boxplot(log(b6) ~ scene+t, data=points, ylab='log(b6)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b6.png">

#### Канал B7

Очень похож на 6-й канал, но вариативность данных внутри классов в нем чуть меньше.

In [14]:
png("Img/CMASK/scene_type_vs_b7.png", width=728, height=728, units="px")
    boxplot(log(b7) ~ scene+t, data=points, ylab='log(b7)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b7.png">

#### Канал B9

В 9-м канале вариативность яркостей облачных пикселей между сценами очень велика. Похоже, что от сцены тут зависит больше, чем от типа объекта.

In [15]:
png("Img/CMASK/scene_type_vs_b9.png", width=728, height=728, units="px")
    boxplot(I(b9^0.5) ~ scene+t, data=points, ylab='sqrt(b9)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b9.png">

#### Канал B10

Аналогичная проблема облачных пикселей и в 10-м канале: от сцены зависит больше, чем от типа объекта, особенно ярко это проявляется для облаков.

In [16]:
png("Img/CMASK/scene_type_vs_b10.png", width=728, height=728, units="px")
    boxplot(b10 ~ scene+t, data=points, ylab='b10')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b10.png">

#### Канал B11

Та же самая проблема: от сцены зависит больше, чем от типа объекта.

In [17]:
png("Img/CMASK/scene_type_vs_b11.png", width=728, height=728, units="px")
    boxplot(b11 ~ scene+t, data=points, ylab='b11')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b11.png">

### Вывод

 * Для различения облаков и снега, видимо, необходимо использовать каналы 1--7. В каналах 9--11 также содержится полезная информация, но воспользоваться ей будет сложнее из-за большей вариативности из-за условий съемки.
 * Каналы 1--5 и каналы 6--7 образуют две группы переменных. Переменные в каждой группе ведут себя схожим образом (поведение каналов 1--5 очень похоже между собой для выбранных классов; аналогично с каналами 6 и 7). Поскольку многие методы очень чувствительны к корреляции объясняющих переменных, то есть смысл подумать о том, чтобы выбрать по одному каналу из каждой группы. Возможно такое уменьшение числа переменных (но и корреляции между ними) даст положительный эффект на точность распознавания.

## Простая модель

В этом разделе построим простейшую модель на основе логистической регрессии и посмотрим, насколько она решает задачу разделения интересующих нас объектов. Пойдем по следующей схеме работы:

1. Предварительный выбор объясняющих переменных. Объясняющих переменных (каналов, в нашем случае) много, но они сильно коррелируют между собой. Для большей устойчивости решения выберем несколько переменных, остальные пока проигнорируем (на следующем этапе оценим правильность выбора переменных).
2. Создание фиктивных переменных. Поскольку у нас выделено три объекта (класс "все остальное" назовем условно одним объектом), и они не упорядочены, то нам понадобится создать фиктивные переменные, которые мы будем использовать.
3. Построение моделей, отражающих связь между объясняющими переменными и зависимыми.
4. Исследуем модели, проверив, выполняются ли условия, на которые опирается процедура построения логистической регрессии.
5. Посмотрим, насколько изменится качество модели, если выбрать другой набор объясняющих переменных.

### Предварительный выбор объясняющих переменных

Посмотрим еще раз на взаимную корреляциию объясняющих переменных (для ускорения отрисовки сделаем небольшую подвыборку данных). 

In [18]:
psample_ind = sample(1:length(points[, 1]), size=500)
psample = points[psample_ind, ]

png("Img/CMASK/all_bands.png", width=728, height=728, units="px")
  pairs(~log(b1)+log(b2)+log(b3)+log(b4)+log(b5)+log(b6)+log(b7)+sqrt(b9)+b10+b11, data=psample, pch=19, cex=0.1) 
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/all_bands.png">

Итак, мы видим, что очень сильна корреляция между 6-м и 7-м каналами, затем между 10-м и 11-м каналами, а также между 1-м, 2-м, 3-м и 4-м каналами, на эти 4 канала довольно похож 5-й, но все-таки, видимо, его следует выделить в отдельную группу. Ту же самую картину мы увидим, если расчитаем коэффициент корреляции между этими переменными:

In [19]:
cor(data.frame(log(points$b1), log(points$b2), log(points$b3), log(points$b4), log(points$b5),
    log(points$b6), log(points$b7), sqrt(points$b9+0.001), points$b10, points$b11))

,log.points.b1.,log.points.b2.,log.points.b3.,log.points.b4.,log.points.b5.,log.points.b6.,log.points.b7.,sqrt.points.b9...0.001.,points.b10,points.b11
log.points.b1.,1.00000000,0.99643580,0.97639013,0.95335102,0.77517809,0.56200970,0.62302423,0.29965771,0.01198254,0.02497259
log.points.b2.,0.99643580,1.00000000,0.99003616,0.97372490,0.81425120,0.60139706,0.66104285,0.30740816,0.01601809,0.02772027
log.points.b3.,0.97639013,0.99003616,1.00000000,0.99513008,0.87490047,0.66889060,0.72503729,0.32957731,0.03250238,0.03846062
log.points.b4.,0.95335102,0.97372490,0.99513008,1.00000000,0.90503815,0.71378830,0.76718334,0.32889596,0.04236531,0.04731348
log.points.b5.,0.7751781,0.8142512,0.8749005,0.9050381,1.0000000,0.8445939,0.8719638,0.3426454,0.1084817,0.1027279
log.points.b6.,0.56200970,0.60139706,0.66889060,0.71378830,0.84459391,1.00000000,0.99188780,0.44287939,0.08266380,0.06304586
log.points.b7.,0.62302423,0.66104285,0.72503729,0.76718334,0.87196375,0.99188780,1.00000000,0.44975283,0.05640044,0.03886842
sqrt.points.b9...0.001.,0.2996577,0.3074082,0.3295773,0.3288960,0.3426454,0.4428794,0.4497528,1.0000000,-0.3242166,-0.4123782
points.b10,0.01198254,0.01601809,0.03250238,0.04236531,0.10848175,0.08266380,0.05640044,-0.32421662,1.00000000,0.98045369
points.b11,0.02497259,0.02772027,0.03846062,0.04731348,0.10272787,0.06304586,0.03886842,-0.41237817,0.98045369,1.00000000


### Фиктивные переменные

Возможно, есть хороший способ создания фиктивных переменных вместо одной зависимой переменной, но я всю процедуру сделаю руками: введу три новых переменных answ1, answ2, answ3 которые будут кодировать тот факт, что пиксель принадлежит облачному, снежному или иному участку снимка соответственно. Для этого создам новый набор данных, в котором будут содержаться только интересующие нас переменные.

In [20]:
pdata = data.frame(answ1=(points$t=='Cloud'), answ2=(points$t=='Snow'), answ3=(points$t=='Other'), 
                   b1=points$b1, b2=points$b2, b3=points$b3, b4=points$b4, b5=points$b5, b6=points$b6, 
                   b7=points$b7, b9=points$b9, b10=points$b10, b11=points$b11, scene=points$scene
                  )

### Построение моделей

Построим три модели (по числу фиктивных переменных) на базе логистической регрессии. Также не будем использовать преобразование переменных на этом этапе построения моделей. Начанем с определения того, что пиксель является облаком.

#### Облако-не-облако

Построим модель определяющую, явлеяется ли данный пиксель облаком и найдем те каналы, которые дают наибольшее качество модели (как отмечалось выше, среди имеющихся объясняющих переменных очень высокая корреляция). Для начала построим модель со всеми имеющимися переменными, включая название сцены.

In [21]:
simple.glm1 = glm(answ1~b1 + b2 + b3 + b4 + b5 + b6 +b7 + b9 + b10 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm1)

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b9 + 
    b10 + b11 + scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3471  -0.0178  -0.0075  -0.0043   3.7550  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -9.2134    12.6052  -0.731  0.46483    
b1            3.8466    36.9262   0.104  0.91703    
b2           14.6088    37.8190   0.386  0.69929    
b3          -99.3805    39.7492  -2.500  0.01241 *  
b4           85.4198    27.3904   3.119  0.00182 ** 
b5          -37.1751     3.9036  -9.523  < 2e-16 ***
b6          -10.9699    12.0687  -0.909  0.36337    
b7          112.6315    14.7005   7.662 1.83e-14 ***
b9           52.7175     8.2705   6.374 1.84e-10 ***
b10          -0.1721     0.1278  -1.347  0.17794    
b11           0.1736     0.1287   1.349  0.17749    
sceneS2       4.5265     0.5016   9.023  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’

Оценкой качества модели может служить несколько параметров, например отклонения (остатки, Deviance) прогнозируемых величин от тех, что зафиксированы на самом деле, другая оценка качества - Akaike Information Criterion (AIC), в котором учитывается как качество подгонки модели, так и ее сложность.  Если оринтироваться на Akaike Information Criterion, то AIC коэффициент у simple.glm2 равен 1313.2 (чем меньше, тем лучше). Но мы видим, что часть коэффициентов регрессии статистически не значимы.

Последовательно будем избавляться от "лишних" коэффициентов регрессии. Для этого в цикле будем удалять по одному коэффициенту и оценивать, как изменился результат. Качество будем оценивать по отклонениям прогнозных значений от реальных. Однако, поскольку у полной модели больше коэффициентов в уравнении регрессии, то и отклонения полной модели будут меньше. Поэтому значимость определенного коэффициента, будем определять по следующей схеме: 
 * выкинем одну переменную, например, b1;
 * очевидно, что в модели без этой переменной (т.е. с коэффициентом b1 == 0) отклонения должны быть больше, чем в модели с этим коэффициентом, поскольку в полной модели $p_2 = 11$ параметров, а в "урезанной" модели $p_1=10$ параметров;
 * соответственно мы можем выдвинуть гипотезу, что коэффициент перед b1 равен нулю и проверить ее, сравнив величины отклонений этих двух моделей: чем больше разность отклонений моделей, тем больше вероятность того, что коэффициент перед b1 ненулевой;
 * разность отклонений двух моделей ассимтотически распределена как Хи-квадрат с $p_2 - p_1 = 1$ степенями свободы: $D_1 - D_1 \sim X^2_{p_2 - p_1}$
 * пробежавшись по всем переменным мы можем определить значимость каждой из них.
 
Эту процедуру автоматизирует функция drop1. Воспользуемся ей:

In [22]:
drop1(simple.glm1, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1289.236,1313.236,NA,NA
b1,1.000000e+00,1.289247e+03,1.311247e+03,1.085342e-02,9.170267e-01
b2,1.0000000,1289.3857572,1311.3857572,0.1494317,0.6990790
b3,1.000000e+00,1.295625e+03,1.317625e+03,6.388735e+00,1.148469e-02
b4,1.000000e+00,1.299156e+03,1.321156e+03,9.919714e+00,1.635178e-03
b5,1.000000e+00,1.394102e+03,1.416102e+03,1.048654e+02,1.307192e-24
b6,1.0000000,1290.0648232,1312.0648232,0.8284977,0.3627073
b7,1.000000e+00,1.353729e+03,1.375729e+03,6.449295e+01,9.687858e-16
b9,1.000000e+00,1.336635e+03,1.358635e+03,4.739900e+01,5.791226e-12
b10,1.0000000,1291.0570998,1313.0570998,1.8207743,0.1772211


Итого, у модели, содержащей все переменные, остатки (Deviance) равняются 1289.236. Если мы викинем b1, то остатки будут равны 1.289247e+03(=1289.247), разность остатков составляет 1.085342e-02=0.01085, соотвественно p-значение распределения Хи-квадрат с одной степенью свободы равно 9.170267e-01, что явно очень много и мы можем выкинуть эту переменную безбоязненно.

Повторяем данную процедуру для нового набора переменных:

In [23]:
simple.glm2 = glm(answ1~ b2 + b3 + b4 + b5 + b6 +b7 + b9 + b10 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm2)
drop1(simple.glm2, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b6 + b7 + b9 + b10 + 
    b11 + scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3477  -0.0178  -0.0075  -0.0043   3.7519  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -9.1119    12.5661  -0.725 0.468381    
b2           18.2279    14.9870   1.216 0.223889    
b3          -97.4374    35.1160  -2.775 0.005525 ** 
b4           83.7884    22.4958   3.725 0.000196 ***
b5          -37.2842     3.7608  -9.914  < 2e-16 ***
b6          -10.7796    11.9278  -0.904 0.366136    
b7          112.3931    14.5178   7.742 9.81e-15 ***
b9           52.7780     8.2485   6.399 1.57e-10 ***
b10          -0.1730     0.1275  -1.357 0.174754    
b11           0.1745     0.1284   1.360 0.173874    
sceneS2       4.5337     0.4969   9.124  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family take

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1289.247,1311.247,NA,NA
b2,1.000000,1290.736493,1310.736493,1.489314,0.222323
b3,1.000000e+00,1.296720e+03,1.316720e+03,7.472685e+00,6.264206e-03
b4,1.000000e+00,1.302318e+03,1.322318e+03,1.307111e+01,2.998858e-04
b5,1.000000e+00,1.403461e+03,1.423461e+03,1.142136e+02,1.170027e-26
b6,1.0000000,1290.0659227,1310.0659227,0.8187438,0.3655477
b7,1.000000e+00,1.354922e+03,1.374922e+03,6.567495e+01,5.317747e-16
b9,1.000000e+00,1.337127e+03,1.357127e+03,4.787991e+01,4.531409e-12
b10,1.0000000,1291.0949258,1311.0949258,1.8477469,0.1740457
b11,1.0000000,1291.1067768,1311.1067768,1.8595978,0.1726712


AIC улучшилось, но остаются еще незначимые коэффициенты. Выкидываем b6 и продолжаем:

In [24]:
simple.glm3 = glm(answ1~ b2 + b3 + b4 + b5 +b7 + b9 + b10 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm3)
drop1(simple.glm3, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b7 + b9 + b10 + b11 + 
    scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3163  -0.0181  -0.0077  -0.0044   3.7809  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -7.0150    12.3120  -0.570 0.568830    
b2           19.0142    14.9371   1.273 0.203035    
b3          -96.5989    35.0669  -2.755 0.005875 ** 
b4           83.9253    22.4924   3.731 0.000191 ***
b5          -38.2918     3.5976 -10.644  < 2e-16 ***
b7           99.8653     4.0618  24.586  < 2e-16 ***
b9           51.7107     8.1173   6.370 1.88e-10 ***
b10          -0.1684     0.1272  -1.324 0.185347    
b11           0.1612     0.1273   1.266 0.205495    
sceneS2       4.4812     0.4922   9.104  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees 

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1290.066,1310.066,NA,NA
b2,1.0000000,1291.6942095,1309.6942095,1.6282868,0.2019402
b3,1.000000e+00,1.297433e+03,1.315433e+03,7.366960e+00,6.643313e-03
b4,1.000000e+00,1.303180e+03,1.321180e+03,1.311393e+01,2.931089e-04
b5,1.000000e+00,1.424451e+03,1.442451e+03,1.343853e+02,4.499798e-31
b7,1.000,4899.839,4917.839,3609.773,0.000
b9,1.000000e+00,1.337128e+03,1.355128e+03,4.706173e+01,6.878583e-12
b10,1.000000,1291.825125,1309.825125,1.759202,0.184724
b11,1.000000,1291.676106,1309.676106,1.610184,0.204466
scene,1.000000e+00,1.396101e+03,1.414101e+03,1.060354e+02,7.243042e-25


Выкидываем b11:

In [25]:
simple.glm4 = glm(answ1~ b2 + b3 + b4 + b5 +b7 + b9 + b10 + scene, data=pdata, family=binomial)
summary(simple.glm4)
drop1(simple.glm4, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b7 + b9 + b10 + scene, 
    family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.2828  -0.0182  -0.0079  -0.0045   3.8098  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -4.43754   12.15280  -0.365 0.715003    
b2            21.35727   14.89459   1.434 0.151602    
b3          -100.04779   35.09888  -2.850 0.004366 ** 
b4            85.05952   22.54604   3.773 0.000161 ***
b5           -38.14592    3.59761 -10.603  < 2e-16 ***
b7            99.46042    4.05353  24.537  < 2e-16 ***
b9            47.12670    7.24054   6.509 7.58e-11 ***
b10           -0.01923    0.04797  -0.401 0.688521    
sceneS2        4.49335    0.49467   9.084  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1291.7  

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1291.676,1309.676,NA,NA
b2,1.0000000,1293.7482433,1309.7482433,2.0721370,0.1500112
b3,1.000000e+00,1.299534e+03,1.315534e+03,7.857899e+00,5.059917e-03
b4,1.000000e+00,1.305059e+03,1.321059e+03,1.338269e+01,2.539573e-04
b5,1.000000e+00,1.424819e+03,1.440819e+03,1.331431e+02,8.412265e-31
b7,1.000,5055.419,5071.419,3763.743,0.000
b9,1.000000e+00,1.343497e+03,1.359497e+03,5.182112e+01,6.079422e-13
b10,1.0000000,1291.8368792,1307.8368792,0.1607729,0.6884459
scene,1.000000e+00,1.397027e+03,1.413027e+03,1.053510e+02,1.023088e-24


Выкидываем b10:

In [26]:
simple.glm5 = glm(answ1~ b2 + b3 + b4 + b5 +b7 + b9 + scene, data=pdata, family=binomial)
summary(simple.glm5)
drop1(simple.glm5, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b7 + b9 + scene, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.2833  -0.0181  -0.0078  -0.0044   3.8032  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -9.2983     0.8449 -11.005  < 2e-16 ***
b2           21.1895    14.8760   1.424 0.154329    
b3          -99.2954    35.0346  -2.834 0.004594 ** 
b4           84.8760    22.5369   3.766 0.000166 ***
b5          -38.4694     3.5044 -10.977  < 2e-16 ***
b7           99.6230     4.0391  24.665  < 2e-16 ***
b9           48.4723     6.4534   7.511 5.86e-14 ***
sceneS2       4.3921     0.4248  10.340  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1291.8  on 50531  degrees of freedom
AIC: 1307.8

Number of Fisher Scoring iterations: 

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1291.837,1307.837,NA,NA
b2,1.0000000,1293.8817588,1307.8817588,2.0448796,0.1527191
b3,1.000000e+00,1.299609e+03,1.313609e+03,7.772014e+00,5.306184e-03
b4,1.000000e+00,1.305183e+03,1.319183e+03,1.334640e+01,2.589188e-04
b5,1.000000e+00,1.440487e+03,1.454487e+03,1.486501e+02,3.420042e-34
b7,1.000,5110.063,5124.063,3818.226,0.000
b9,1.000000e+00,1.362258e+03,1.376258e+03,7.042144e+01,4.789659e-17
scene,1.000000e+00,1.448253e+03,1.462253e+03,1.564162e+02,6.866208e-36


Избавляемся от незначимого коэффициента b2:

In [27]:
simple.glm6 = glm(answ1~ b3 + b4 + b5 +b7 + b9 + scene, data=pdata, family=binomial)
summary(simple.glm6)
drop1(simple.glm6, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b3 + b4 + b5 + b7 + b9 + scene, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.2517  -0.0178  -0.0077  -0.0044   3.7779  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -8.6519     0.7169 -12.069  < 2e-16 ***
b3          -58.9995    21.2553  -2.776 0.005507 ** 
b4           69.9411    20.2974   3.446 0.000569 ***
b5          -39.8046     3.3812 -11.772  < 2e-16 ***
b7           99.1685     4.0099  24.731  < 2e-16 ***
b9           48.9073     6.4829   7.544 4.56e-14 ***
sceneS2       4.4688     0.4258  10.494  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1293.9  on 50532  degrees of freedom
AIC: 1307.9

Number of Fisher Scoring iterations: 11


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1293.882,1307.882,NA,NA
b3,1.000000e+00,1.301168e+03,1.313168e+03,7.286727e+00,6.946592e-03
b4,1.000000e+00,1.305237e+03,1.317237e+03,1.135539e+01,7.522941e-04
b5,1.000000e+00,1.492950e+03,1.504950e+03,1.990681e+02,3.335693e-45
b7,1.000,5114.362,5126.362,3820.481,0.000
b9,1.000000e+00,1.367413e+03,1.379413e+03,7.353171e+01,9.903246e-18
scene,1.000000e+00,1.462616e+03,1.474616e+03,1.687340e+02,1.398566e-38


Больше удалять нечего. Эта модель -- максимум точности, которой мы можем достичь при анализе. К сожалению, среди значимых коэффициентов осталась переменная scene, что означает, что в модели должен фигурировать идентификатор сцены. Автоматически это означает, что для каждой сцены, по сути, мы должы будем строить свою модель. Альтернативный вариант -- использование смешанных моделей (mixed effects), что даст значительно усложнение всех вычислений.

Итак, надежда на то, что сцена окажется не значимой, и можно построить модель, в которой не фигурирует идентификатор сцены, не оправдалась. Вернемся на шаг назад, вручную удалим все переменные, которые зависят от сцены (см. первый этап анализа), затем повторим процедуру отбора оптимальной модели и посмотрим, насколько ухудшадся показатели такой модели по сравнению с simple.glm6. 

Ниже повторяется вся процедура отбора переменных без дополнительных комментарив:

In [28]:
simple.glm7 = glm(answ1~b1 + b2 + b3 + b4 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm7)
drop1(simple.glm7, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3647  -0.0290  -0.0156  -0.0091   4.3224  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -15.105      1.288 -11.725  < 2e-16 ***
b1           102.756     36.813   2.791  0.00525 ** 
b2           -65.801     40.518  -1.624  0.10437    
b3           -70.817     33.606  -2.107  0.03509 *  
b4            47.993     21.618   2.220  0.02642 *  
b5           -30.658      3.171  -9.670  < 2e-16 ***
b6            34.449     11.312   3.045  0.00232 ** 
b7            80.444     13.096   6.142 8.13e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1438.2  on 50531  degrees of freedom
AIC: 1454.2

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1438.155,1454.155,NA,NA
b1,1.000000e+00,1.446026e+03,1.460026e+03,7.871087e+00,5.023144e-03
b2,1.000000,1440.805798,1454.805798,2.651043,0.103482
b3,1.000000e+00,1.442571e+03,1.456571e+03,4.416502e+00,3.559292e-02
b4,1.000000e+00,1.443011e+03,1.457011e+03,4.855902e+00,2.755171e-02
b5,1.000000e+00,1.545974e+03,1.559974e+03,1.078194e+02,2.944222e-25
b6,1.000000e+00,1.447446e+03,1.461446e+03,9.291302e+00,2.302445e-03
b7,1.000000e+00,1.477756e+03,1.491756e+03,3.960169e+01,3.114140e-10


In [29]:
simple.glm8 = glm(answ1~b1 + b3 + b4 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm8)
drop1(simple.glm8, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b3 + b4 + b5 + b6 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3814  -0.0291  -0.0158  -0.0093   4.3174  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -13.943      1.065 -13.093  < 2e-16 ***
b1            47.000     13.733   3.422 0.000621 ***
b3           -63.187     33.583  -1.882 0.059902 .  
b4            29.917     18.754   1.595 0.110650    
b5           -31.536      3.131 -10.073  < 2e-16 ***
b6            38.963     10.989   3.545 0.000392 ***
b7            75.563     12.743   5.930 3.04e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1440.8  on 50532  degrees of freedom
AIC: 1454.8

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1440.806,1454.806,NA,NA
b1,1.000000e+00,1.457519e+03,1.469519e+03,1.671286e+01,4.348525e-05
b3,1.000000e+00,1.444625e+03,1.456625e+03,3.819134e+00,5.067041e-02
b4,1.0000000,1443.3783405,1455.3783405,2.5725424,0.1087329
b5,1.000000e+00,1.558526e+03,1.570526e+03,1.177206e+02,1.996197e-27
b6,1.000000e+00,1.453345e+03,1.465345e+03,1.253926e+01,3.984911e-04
b7,1.000000e+00,1.477811e+03,1.489811e+03,3.700510e+01,1.178204e-09


In [30]:
simple.glm9 = glm(answ1~b1 + b3 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm9)
drop1(simple.glm9, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b3 + b5 + b6 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3205  -0.0293  -0.0163  -0.0096   4.2957  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -13.513      1.000 -13.512  < 2e-16 ***
b1            32.036      9.411   3.404 0.000664 ***
b3           -13.358     10.785  -1.239 0.215518    
b5           -32.009      3.074 -10.411  < 2e-16 ***
b6            42.514     10.723   3.965 7.34e-05 ***
b7            70.289     12.227   5.749 8.99e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1443.4  on 50533  degrees of freedom
AIC: 1455.4

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1443.378,1455.378,NA,NA
b1,1.000000e+00,1.457523e+03,1.467523e+03,1.414423e+01,1.693160e-04
b3,1.0000000,1444.9792568,1454.9792568,1.6009163,0.2057734
b5,1.000000e+00,1.567276e+03,1.577276e+03,1.238977e+02,8.869957e-29
b6,1.000000e+00,1.459115e+03,1.469115e+03,1.573627e+01,7.281458e-05
b7,1.000000e+00,1.477851e+03,1.487851e+03,3.447230e+01,4.323603e-09


In [31]:
simple.glm10 = glm(answ1~b1 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm10)
drop1(simple.glm10, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b5 + b6 + b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3348  -0.0290  -0.0160  -0.0096   4.2994  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -12.4185     0.4587 -27.073  < 2e-16 ***
b1           20.8339     2.6116   7.977 1.49e-15 ***
b5          -34.7703     2.1604 -16.095  < 2e-16 ***
b6           45.3291    10.4937   4.320 1.56e-05 ***
b7           67.9584    12.0931   5.620 1.91e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14054  on 50538  degrees of freedom
Residual deviance:  1445  on 50534  degrees of freedom
AIC: 1455

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1444.979,1454.979,NA,NA
b1,1.000000e+00,1.514534e+03,1.522534e+03,6.955486e+01,7.431837e-17
b5,1.000000e+00,2.008890e+03,2.016890e+03,5.639107e+02,1.185530e-124
b6,1.000000e+00,1.463680e+03,1.471680e+03,1.870055e+01,1.529382e-05
b7,1.000000e+00,1.477961e+03,1.485961e+03,3.298191e+01,9.302018e-09


Итак, в этом случае оптимальный набор переменных: b1, b5, b6, b7. AIC: 1455 (для сравнения: AIC у simple.glm6 равен 1307.9).

##### Валидация модели

Посмотрим, нет ли каких-либо закономерностей в ответах. Для этого построим график зависимостей остатков от подогнанных значений.

Для оценки качества модели изучим поведение зависимости остатков (невязка модели) от прогнозируемой величины и от независимых переменных. В идеале остатки не должны показывать каких-либо закономерностей.

In [32]:
E10 = resid(simple.glm10, type="response")
png("Img/CMASK/glm10_res_fitted.png", width=728, height=728, units="px")
  plot(simple.glm10$fitted.values, E10, cex=0.1)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm10_res_fitted.png">

Поскольку ответ логистической регрессии это один или ноль, а значений, использующихся для подгонки достаточно много, то на графике видим две полосы точек (остатков). В итогое понять, есть ли закономерность в остатках становится сложно. Поэтому упорядочим полученные ответы по возрастанию fitted.values, разобьем их на группы и посчитаем в каждой группе среднее значение невязки (средний остаток).

In [33]:
ER10 = data.frame(err=E10, fitted=simple.glm10$fitted.values, 
                b1=pdata$b1, b2=pdata$b2, b3=pdata$b3, b4=pdata$b4, b5=pdata$b5, b6=pdata$b6, 
                b7=pdata$b7, b9=pdata$b9, b10=pdata$b10, b11=pdata$b11, scene=pdata$scene)

# Сортируем по порядку прогнозируемых величин
ER10 = ER10[order(ER10$fitted), ]

# Разбиваем на группы
group_size = 100
ER10 = cbind(id=(1:length(ER10$fitted))%/% group_size, ER10)
# Вычислим среднее значение ошибки в каждой группе и поместим в таблицу
lookup_table = aggregate(ER10['err'], by=list(ER10$id), FUN=mean)
names(lookup_table) = c('id', 'err')

# Создадим вектор из средний значений ошибок 
a = sapply(ER10$id, function(id){lookup_table[lookup_table$id==id, 'err']})

ER10 = cbind(ER10, mean.err=a)


png("Img/CMASK/glm10_res_fitted_srt.png", width=728, height=728, units="px")
  plot(ER10$fitted, ER10$mean.err, cex=0.1)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm10_res_fitted_srt.png">

Как видим, в усредненных остатках есть некоторая тенденция: если прогнозируемое значение близко к нулю (не облако), то остаток скорее будет отцательным, для средних прогнозируемых значений остатки чаще положительны, а если прогнозируемое значение близко к единице, то отстаток опять будет скорее отрицательным. Для идеально подогнанной кривой усредненные остатки должны быть более-менее равномерно разбросаны вокруг нуля и не зависеть от прогнозируемой величины.

Аналогично, построим графики зависимостей остатков от всех переменных (как использованных в модели, так и не использованных).

In [34]:
png("Img/CMASK/glm10_res_bXX_srt.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    plot(ER10$b1, ER10$mean.err, cex=0.1)
    plot(ER10$b2, ER10$mean.err, cex=0.1)
    plot(ER10$b3, ER10$mean.err, cex=0.1)
    plot(ER10$b4, ER10$mean.err, cex=0.1)

    plot(ER10$b5, ER10$mean.err, cex=0.1)
    plot(ER10$b6, ER10$mean.err, cex=0.1)
    plot(ER10$b7, ER10$mean.err, cex=0.1)

    plot(ER10$b9, ER10$mean.err, cex=0.1)

    plot(ER10$b10, ER10$mean.err, cex=0.1)
    plot(ER10$b11, ER10$mean.err, cex=0.1)

    boxplot(ER10$mean.err ~ ER10$scene, cex=0.1)

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm10_res_bXX_srt.png">

Каких-то бросающихся в глаза тенденций (например, низких значений остатков при высоких величинах переменных или чего-нибудь в этом роде) в зависимости остатков от каналов не заметно: отрицательные остатки обычно уравновешиваются положительными. Также хорошая новость в том, что остатки не показывают зависимости от сцены.

In [35]:
library(mgcv)

Loading required package: nlme
This is mgcv 1.7-28. For overview type 'help("mgcv-package")'.
